## Use this script to validate individual organizations for data QC purposes

In [1]:
# importing libraries
import pandas as pd
import os
from datetime import datetime
import shutil
import glob

In [ ]:

# Create a file path to the two directories where the input file is stored
input_path = r'C:\\Users\\azona\\OneDrive - Cerity Partners\\Portfolio Management Team\\2 - Monthly AUM and Performance\\DEV\\2.1.1 - DEV Addepar Queries'

# Setting the directory then importing a file
os.chdir(input_path)
input_file_name = 'Holdings Cleveland.xlsx'
data = pd.read_excel(input_file_name)

In [ ]:
# Establishing values that should not be rolled up
values_no_roll_up = ['Unassigned', 'Roll up - Non SMA']

# Getting a cummulative count by SMA ID
data['row_count'] = data.groupby('SMA ID').cumcount() + 1

In [5]:
# First, this code filters down to the rows that have a "SMA ID" that should not be rolled up and filtering down to the first row, which is the top of the rollup in the Excel doc
# Then, it goes to the values in the "no roll up" variable and removes the top of the roll up, the first row, and keeps the rest
data = data[((~data['SMA ID'].isin(values_no_roll_up)) & (data['row_count'] == 1 ) | (data['SMA ID'].isin(values_no_roll_up)) & (data['row_count'] != 1) ) ]

# Getting rid of the total row at the bottom
data = data[data['CP SMA ID'] != 'Total']

In [6]:
# Removing file extention from the file name
clean_file_name = input_file_name.replace('.xlsx', '')

# Getting the organization name from the file name
organization_name = clean_file_name.split('Holdings ')[-1]

# Adding organization name to the dataset
data['Organization'] = '*' + organization_name

In [ ]:
# Establishing the columns to drop
columns_2_drop = ['SMA ID', 'row_count']

# Removing columns that are not needed in the final output
data = data.drop(columns = columns_2_drop)

# Sorting the columns in the sequence consistent with the way the output is to be seen
columns_sequence = ['Organization' , 'CP SMA ID', 'Adjusted Value (No Div, USD)', 'Adjusted Net Cash Flow (YTD, No Div, USD)']
data = data[columns_sequence]

In [ ]:
# use this to check row counts if you would like against other results in the batch file
data

,Organization,CP SMA ID,"Adjusted Value (No Div, USD)","Adjusted Net Cash Flow (YTD, No Div, USD)"
0,*Cleveland,AQR1,9.289159e+06,8.968512e+06
903,*Cleveland,Arist1,6.395827e+06,-5.758577e+06
959,*Cleveland,Arist2,9.520457e+06,-3.000000e+05
1026,*Cleveland,BR3,4.016062e+06,1.466417e+06
1179,*Cleveland,Clear1,7.774646e+05,-3.637979e-12
...,...,...,...,...
5291,*Cleveland,9887293,1.000000e+05,1.000000e+05
5292,*Cleveland,9897935,3.807788e+04,-1.525050e+03
5293,*Cleveland,9951722,3.239240e+05,0.000000e+00
5294,*Cleveland,9989748,1.007396e+04,-1.625000e+02


In [9]:
# Establishing output path locations
output_path = 'C:\\Users\\azona\\OneDrive - Cerity Partners\\Portfolio Management Team\\2 - Monthly AUM and Performance\\DEV\\2.1.1 - DEV Addepar Queries\\Outputs'
output_file = os.path.join(output_path, "testing.xlsx")
data.to_excel(output_file, sheet_name='sheet1', index=False)